In [ ]:
#load in libraries
library(shiny)
library("geojsonio")
library(sp)
library(broom)
library(ggplot2)
library(ggmap)
library(maps)
library(mapdata)
library(sf)
library(tidyverse)
library(lubridate)
library(zoo)
library(forecast)
library(shinythemes)
library(shinybusy)
library(grid)
library(gtable)


ui <- fluidPage(
    
    theme = shinytheme("simplex"),
    titlePanel("CA Wildfire Visualizations"),
    
    navbarPage(
        "Visualizations to Explore",
        
        tabPanel(
            "Fire Map",
            sidebarLayout(
                sidebarPanel(
                    width = 3,
                    
                    sliderInput(
                        inputId = "year",
                        label = "select year to display:",
                        min = 1950,
                        max = 2019,
                        value = 2017,
                        sep = ''
                    )
                ),
                
                mainPanel(
                    plotOutput(
                        "year.map", 
                        height="600px", 
                        width="100%"
                    )
                )
            )
        ),
        
        tabPanel(
            "Acres Burned Over Time",
            sidebarLayout(
                sidebarPanel(
                    width = 3,
                    
                    sliderInput(
                        inputId = "ts.range",
                        label = "select range for time series:",
                        min = 1950,
                        max = 2019,
                        value = c(1993, 2017),
                        sep = ''
                    ),
                    
                    selectInput(inputId = "smooth", 
                        label = "select window for smoothing the trend:", 
                        choices = list("1 year" = 13, "2 years" = 25, "3 years" = 37, "4 years" = 49, "5 years" = 61, "6 years" = 73, "7 years" = 85), 
                        selected = 49
                    )
                ),
                
                mainPanel(
                    plotOutput(
                        "decomposition", 
                        height="600px", 
                        width="100%"
                    )
                )
            )
        ),
        
        tabPanel(
            "Seasonal Patterns",
            sidebarLayout(
                sidebarPanel(
                    width = 3,
                    
                    sliderInput(
                        inputId = "seas.range",
                        label = "select range years to display:",
                        min = 1950,
                        max = 2019,
                        value = c(1993, 2017),
                        sep = ''
                    )
                ),
                
                mainPanel(
                    plotOutput(
                        "seasons", 
                        height="600px", 
                        width="100%"
                    )
                )
            )
        ),
        
        tabPanel(
            "Forecasting",
            sidebarLayout(
                sidebarPanel(
                    width = 3,
                    
                    selectInput(inputId = "model", 
                        label = "select model for forecasting:", 
                        choices = list("ARIMA" = 'arima', "exponential smoothing" = 'ets', "feed-forward neural network" = 'nn'), 
                        selected = 'arima'
                    )  
                ),
                
                mainPanel(
                    plotOutput(
                        "forecast", 
                        height="590px", 
                        width="100%"
                    ),
                    
                    uiOutput(
                        "mae"
                    )
                      
                )
            )
        ),
        
        tabPanel(
            "References",
            sidebarLayout(
                sidebarPanel(
                    width = 3,
                    h3("thanks for visualizing!")
                ),
                
                mainPanel(
                    h4("data sourced from ", a("here.", href = "https://gis.data.ca.gov/datasets/CALFIRE-Forestry::california-fire-perimeters-1950?orderBy=YEAR_")), 
                    h4("source code here."),
                    h4("a couple of nice references for time series analysis in R, here and here.")
                )
            )
        )
        
    )
)

server <- function(input, output) {
    show_modal_progress_line()
    update_modal_progress(0.05)
    
    #load in data
    cafp <- geojson_read("./ca_fire_perimeters.geojson", what = "sp")
    update_modal_progress(0.45)
    
    #clean data
    out_of_range <- cafp@data[cafp@data$ALARM_DATE < "1950-01-01" | cafp@data$ALARM_DATE > "2020-01-01" ,]
    out_of_range <- out_of_range[complete.cases(out_of_range[ , 1:3 ]), ]
    
    cafp@data[ 13956, 'ALARM_DATE'] = "2016-09-26"
    cafp@data[ 14900, 'ALARM_DATE'] = "2018-11-13"
    cafp@data[ 15434, 'ALARM_DATE'] = "2019-05-29"
    cafp@data[ 15434, 'CONT_DATE'] = "2019-05-29"
    
    cafp@data$YEAR <- as.Date(paste(cafp@data$YEAR_, 1, 1, sep = "-"))
    cafp@data$MONTH_YEAR <- as.Date(cut(cafp@data$ALARM_DATE, 'month'))
    
    #create timeseries of average daily acres burned, per month, from 1950 to 2019
    acres.monthly <- cafp@data[ complete.cases(cafp@data$MONTH_YEAR), ] %>%
                        group_by(MONTH_YEAR) %>%
                        summarize(total_acres_burned = sum(GIS_ACRES, na.rm = TRUE)) %>%
                        read.zoo() %>%
                        aggregate(as.yearmon, sum) %>%
                        as.zooreg(freq = 12) %>%
                        as.ts() %>%
                        na.fill(fill = 0)
    acres.perday <- acres.monthly/monthdays(acres.monthly)
    acres.perday1 <- acres.perday + 1
    
    #create base CA map
    states <- map_data("state")
    ca_df <- subset(states, region == "california")
    ca_base <- ggplot(data = ca_df, mapping = aes(x = long, y = lat, group = group)) + 
        coord_map() + 
        geom_polygon(color = "black", fill = "gray") +
        theme_dark()

    #transform timeseries with box cox
    lambda <- BoxCox.lambda(acres.perday1)
    acres.perday1.boxcox <- acres.perday1 %>% BoxCox(lambda)
    perday1.boxcox.decomp <- acres.perday1.boxcox %>% mstl()
    
    #generate fiery color palette
    fire.pal <- colorRampPalette(c("darkred", "red","orange", "gold"))
    
    #subset training set for measuring MAE to evaluate forecasts
    train = window(acres.perday1, start = 1990, end = c(2014, 12))
    h = 60
    update_modal_progress(0.63)
    
    #generate forecasts
    arima <- forecast(auto.arima(acres.perday1, lambda = lambda, biasadj = TRUE), h = h)
    ets <- forecast(ets(acres.perday1, lambda = lambda, biasadj = TRUE), h = h)
    nn <- forecast(nnetar(acres.perday1, lambda = lambda, biasadj = TRUE), h = h)
    update_modal_progress(0.84)
    
    #forecast based on training set, to measure MAE and evaluate forecasts
    arima.train <- forecast(auto.arima(train, lambda = lambda, biasadj = TRUE), h = h)
    ets.train <- forecast(ets(train, lambda = lambda, biasadj = TRUE), h = h)
    nn.train <- forecast(nnetar(train, lambda = lambda, biasadj = TRUE), h = h)
    update_modal_progress(0.98)
    
    fmodel <- reactive({
        if (input$model == 'arima') {
            rmodel <- arima
        } else if (input$model == 'ets') {
            rmodel <- ets
        } else if (input$model == 'nn') {
            rmodel <- nn
        }
    })
    
    fmodel.train <- reactive({
        if (input$model == 'arima') {
            rmodel.train <- arima.train
        } else if (input$model == 'ets') {
            rmodel.train <- ets.train
        } else if (input$model == 'nn') {
            rmodel.train <- nn.train
        }
    })
    
    #plot wildfire polygons, per year
    output$year.map <- renderPlot({
        cafp_subset <- subset(cafp, YEAR_ == input$year)
        ca_base + 
            geom_polygon(data = cafp_subset, aes( x = long, y = lat, group = group), fill="red", color="red") +
            ggtitle(paste("wildfires in", input$year)) +
            labs(y="latitude", x = "longitude") +
            theme(plot.title = element_text(size = (18)))  
    })
    remove_modal_progress()
    
    #plot timeseries decomposition
    output$decomposition <- renderPlot({
        decomposition.df <- cbind(daily.avg = acres.perday,
                            trend = InvBoxCox(trendcycle(perday1.boxcox.decomp), lambda),
                            trend.smoothed = ma(InvBoxCox(trendcycle(perday1.boxcox.decomp), lambda), order = strtoi(input$smooth)),
                            transformed = BoxCox(acres.perday1,lambda),
                            seasonal.tr = seasonal(perday1.boxcox.decomp),
                            remainder.tr = remainder(perday1.boxcox.decomp))
        autoplot(window(decomposition.df, start = input$ts.range[1], end = c(input$ts.range[2], 12)), color = "orange", facet=TRUE) +
        xlab("years") + ylab("") +
        ggtitle("acres burned in CA wildfires") +
        theme_dark()
    })
    
    #plot each year of wildfire timeseries, over seasonal axis
    output$seasons <- renderPlot({
        ggseasonplot(window(acres.perday1.boxcox, start = input$seas.range[1], end = c(input$seas.range[2], 12))) +
            ggtitle('seasonal variation in CA wildfires') +
            xlab('month') + ylab('daily average acres burned (transformed)') +
            theme_dark() +
            scale_color_manual(values = c(fire.pal(input$seas.range[2] - input$seas.range[1] + 1)))
    })
    
    #plot original timeseries, extended with five year forecast
    output$forecast <- renderPlot({
        transformed <- autoplot(acres.perday1.boxcox) +
            autolayer(BoxCox(fmodel()$mean, lambda = lambda), series = 'forecast', color = "orange") +
            xlab('year') + ylab('') +
            ggtitle('transformed') +
            theme_dark()
        raw <- autoplot(acres.perday1) +
            autolayer(fmodel()$mean, series = 'forecast', color = "red") +
            xlab('year') + ylab('') +
            ggtitle('daily average acres burned') +
            theme_dark()

        plots <- rbind(ggplotGrob(raw), ggplotGrob(transformed), size = "first")
        grid.draw(plots)
    })
    
    #report MAE for selected forecast
    output$mae <- renderUI({
        error <-  fmodel.train() %>% accuracy(acres.perday1)
        h3("MAE: ", round(error["Test set", "MAE"], 2))
    })
    
}

shinyApp(ui = ui, server = server)


Listening on http://127.0.0.1:3221

Regions defined for each Polygons

